# Index Articles and Test Search

Interactive notebook for indexing articles and testing search performance.

In [ ]:
import json
import requests
import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import concurrent.futures
from datetime import datetime, timedelta
import random
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('.env', override=True)

# Configuration
SEARCH_ENGINE_BASE_URL = os.getenv('SEARCH_ENGINE_BASE_URL', 'https://search_api.toy.x.upstage.ai')
UPSTAGE_API_KEY = os.getenv('UPSTAGE_API_KEY')
COLLECTION_NAME = "themilk"
BATCH_SIZE = 20
MAX_WORKERS = 1

if not UPSTAGE_API_KEY:
    raise ValueError("UPSTAGE_API_KEY not found in .env.local file")


print(f"Target: {SEARCH_ENGINE_BASE_URL}")
print(f"Collection: {COLLECTION_NAME}")

Target: https://search_api.toy.x.upstage.ai
Collection: themilk


/Users/hunkim/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. Load Data

In [2]:
articles_path = "./data/articles.json"

print(f"Loading articles from {articles_path}...")
with open(articles_path, 'r', encoding='utf-8') as f:
    articles = json.load(f)
    
print(f"Loaded {len(articles)} articles")

# Preview first article
if articles:
    print("\nSample article:")
    print(json.dumps(articles[0], indent=2, ensure_ascii=False)[:500] + "...")

Loading articles from ./data/articles.json...
Loaded 8730 articles

Sample article:
{
  "id": 3,
  "content": "미국 추수감사절 최대 쇼핑 할인 행사인 '블랙 프라이데이'가 영향력을 잃어간다. 블랙 프라이데이(Black Friday, 이하 블프: 추수감사절 다음 날 시작되는 대규모 할인 행사)라고 하는 대박 할인 행사가 사실상 사라진다. 블프는 유통(리테일) 산업에 상당한 영향을 미치는 행사다. 유통기업은 블프를 계기로 드디어 그 해 흑자로 돌아선다. 미국 최대 백화점 메이시(Macy's)의 제프리 제네트 회장은 \"연말 연시 수요는 일반적으로 추수 감사절과 크리스마스 사이에 발생한다. 하지만 올해는 더 일찍 수요가 시작되고 있다”고 말했다. 추수감사절 시즌에 맞춰 일제히 할인을 하는 블프의 시대는 끝났다. 블프의 매력은 ‘도어버스터(doorbuster: 블랙 프라이데이 당일 선착순으로 구매할 수 있는 할인률 높은 상품들. 소비자들을 매장으로 끌어들이기 위한 미끼 역할을 하는 상품들)’ 할인이다. 추수감사절 당일 저녁부터 줄을 서다가, 월마트(W...


## 2. Prepare & Index Documents

In [3]:
def prepare_document(article):
    """Helper to transform article into indexing format"""
    article_id = article['id']
    content = article['content']
    
    # Use first 100 characters as title
    title = content[:100] if len(content) > 100 else content
    
    # Generate a random date in 2024-2025
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2025, 11, 23)
    random_days = random.randint(0, (end_date - start_date).days)
    date = (start_date + timedelta(days=random_days)).strftime("%Y-%m-%d")
    
    return {
        "title": title,
        "url": f"https://themiilk.com/articles/{article_id}",
        "content": content,
        "date": date
    }

def index_batch(batch_data):
    """Helper to index a single batch"""
    batch, batch_idx = batch_data
    documents = [prepare_document(a) for a in batch]

    index_endpoint = f"{SEARCH_ENGINE_BASE_URL}/index/{COLLECTION_NAME}"   
    
    headers = {
        "Authorization": f"Bearer {UPSTAGE_API_KEY}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.post(
            index_endpoint, 
            headers=headers, 
            json={"documents": documents}, 
            verify=False, 
            timeout=120
        )
        if response.status_code == 200:
            return len(documents), 0, None
        return 0, len(documents), f"Batch {batch_idx}: {response.status_code} - {response.text}"
    except Exception as e:
        return 0, len(documents), f"Batch {batch_idx}: {str(e)}"

In [4]:
# Create batches
batches = []
for i in range(0, len(articles), BATCH_SIZE):
    batches.append((articles[i:i+BATCH_SIZE], i // BATCH_SIZE))

print(f"Created {len(batches)} batches from {len(articles)} articles")

# Run Indexing
print(f"Starting indexing with {MAX_WORKERS} workers...")
start_time = time.time()
indexed_count = 0
failed_count = 0

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(index_batch, b) for b in batches]
    
    for future in concurrent.futures.as_completed(futures):
        success, fail, error = future.result()
        indexed_count += success
        failed_count += fail
        
        if error:
            print(f"✗ {error}")
            
        if (indexed_count + failed_count) % 100 == 0:
            print(f"Progress: {indexed_count + failed_count}/{len(articles)}")

elapsed = time.time() - start_time
print(f"\nDone! Indexed: {indexed_count}, Failed: {failed_count}")
print(f"Time: {elapsed:.2f}s ({indexed_count/elapsed if elapsed > 0 else 0:.2f} docs/s)")

Created 437 batches from 8730 articles
Starting indexing with 1 workers...
Progress: 100/8730
Progress: 200/8730
Progress: 300/8730
Progress: 400/8730
Progress: 500/8730
Progress: 600/8730
Progress: 700/8730
Progress: 800/8730
Progress: 900/8730
Progress: 1000/8730
Progress: 1100/8730
Progress: 1200/8730
Progress: 1300/8730
Progress: 1400/8730
Progress: 1500/8730
Progress: 1600/8730
Progress: 1700/8730
Progress: 1800/8730
Progress: 1900/8730
Progress: 2000/8730
Progress: 2100/8730
Progress: 2200/8730
Progress: 2300/8730
Progress: 2400/8730
Progress: 2500/8730
Progress: 2600/8730
Progress: 2700/8730
Progress: 2800/8730
Progress: 2900/8730
Progress: 3000/8730
Progress: 3100/8730
Progress: 3200/8730
Progress: 3300/8730
Progress: 3400/8730
Progress: 3500/8730
Progress: 3600/8730
Progress: 3700/8730
Progress: 3800/8730
Progress: 3900/8730
Progress: 4000/8730
Progress: 4100/8730
Progress: 4200/8730
Progress: 4300/8730
Progress: 4400/8730
Progress: 4500/8730
Progress: 4600/8730
Progress: 4700

## 3. Test Search

In [6]:
test_queries = [
    "블랙 프라이데이",
    "할인",
    "쇼핑",
    "미국",
    "온라인",
    "소비자",
    "매장",
    "유통",
    "크리스마스",
    "추수감사절"
]

print(f"Testing {len(test_queries)} queries...\n")

headers = {
    "Authorization": f"Bearer {UPSTAGE_API_KEY}",
    "Content-Type": "application/json"
}

search_endpoint = f"{SEARCH_ENGINE_BASE_URL}/search/{COLLECTION_NAME}"

for query in test_queries:
    print(f"Query: '{query}'")
    
    try:
        t0 = time.time()
        response = requests.post(
            search_endpoint, 
            headers=headers, 
            json={"query": query}, 
            verify=False, 
            timeout=60
        )
        dt = time.time() - t0
        
        if response.status_code == 200:
            results = response.json().get('results', [])
            print(f"  ✓ Found {len(results)} results in {dt*1000:.1f}ms")
            if results:
                top = results[0]
                print(f"  Top: {top.get('title', 'No Title')[:50]}... (Score: {top.get('score', 'N/A')})")
        else:
            print(f"  ✗ Failed: {response.status_code} - {response.text}")
            
    except Exception as e:
        print(f"  ✗ Error: {e}")
    
    print()

Testing 10 queries...

Query: '블랙 프라이데이'
  ✓ Found 10 results in 1303.4ms
  Top: 백악관 "테슬라, 다른 차량에 충전소 개방"  미국의 전동화가 빠르게 진행되고 있는데요. ... (Score: 0.5)

Query: '할인'
  ✓ Found 10 results in 409.5ms
  Top: 안녕하세요.여행을 계획할때 항공권 다음으로 지출이 큰 내역은 바로 호텔입니다. 하지만 호텔... (Score: 0.5)

Query: '쇼핑'
  ✓ Found 10 results in 361.5ms
  Top: 안녕하세요. 실리콘밸리의 스타트업 전도사, 더밀크 스타트업 포커스입니다.‘빅데이터’가 세상... (Score: 0.5)

Query: '미국'
  ✓ Found 10 results in 415.1ms
  Top: 미국은 코로나 바이러스로 인해 3월 29일 현재 12만2653명의 확진자와 2112명의 사... (Score: 0.5)

Query: '온라인'
  ✓ Found 10 results in 352.7ms
  Top: AI 기술업체 오픈AI가 숏폼 동영상 플랫폼 ‘틱톡(Tiktok)’ 공식 계정을 개설, 본... (Score: 0.5)

Query: '소비자'
  ✓ Found 10 results in 356.3ms
  Top: 익숙한 얼굴들을 보니 정말 기쁘네요. 여러분의 창의성은 우리 삶을 풍요롭게 합니다. 7일(... (Score: 0.5)

Query: '매장'
  ✓ Found 10 results in 338.5ms
  Top: 

  
  Track all markets on TradingView
  

 📈 오늘의... (Score: 0.5)

Query: '유통'
  ✓ Found 10 results in 346.8ms
  Top: 탈중앙화, 분권화, 소비자 참여라는 속성을 지닌 ‘웹 3.0(Web 3.0)’이 할리우드·... (Score: 0.5)

Query: '크리스마스'
 

## 4. AI Overview with the Milk Index

In [10]:
AI_OVERVIEW_URL = "https://solar-web-search.cosmic.upstage.ai/v1/chat/completions"

payload = {
  "model": "solar-pro2-search",
  "messages": [
    {
      "role": "user",
      "content": "윤송이 박사님 강의"
    }
  ],
  "stream": False,
  "web_search_options": {
    "search_context_size": "medium",
    "external_search_engine_only": True,
    "servers": [
      {
        "url": search_endpoint,
        "key": UPSTAGE_API_KEY
      }
    ]
  }
}

headers = {
  'Authorization': f'Bearer {UPSTAGE_API_KEY}',
  'Content-Type': 'application/json'
}

print(f"Querying Solar API with external search: {search_endpoint}...")
response = requests.post(AI_OVERVIEW_URL, headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))


Querying Solar API with external search: https://search_api.toy.x.upstage.ai/search/themilk...
{
  "id": "6d3552e5-536c-4dcf-8902-62e1b2c7ddf3",
  "object": "chat.completion",
  "created": 1764422960,
  "model": "solar-pro2-search",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "**\uc724\uc1a1\uc774 \ubc15\uc0ac\ub294 AI\ub97c \uc778\ub958 \uc9c4\ud654\uc758 \ubcc0\uace1\uc810\uc73c\ub85c \ubcf4\uba70, \ubd88\uacfc \uc5b8\uc5b4\uc5d0 \uc774\uc740 \uc138 \ubc88\uc9f8 \uadfc\uc6d0\uc801 \ubcc0\ud654 \ub3c4\uad6c\ub85c \uac15\uc870\ud588\uc2b5\ub2c8\ub2e4 [1][4][8].**  \n\n### \uc0c1\uc138 \uc124\uba85  \n1. **AI\ub97c \uc778\ub958 \ubb38\uba85\uc758 \uc138 \ubc88\uc9f8 \ubd88\uae38\ub85c \uaddc\uc815**  \n   - \uc724\uc1a1\uc774 PVP \ub300\ud45c\ub294 2026 \uae30\uc870\uc5f0\uc124\uc5d0\uc11c \"AI\ub294 \ub300\ud55c\ubbfc\uad6d\uc758 \uc138 \ubc88\uc9f8 \ubd88\"\uc774\ub77c\uace0 \uc8fc\uc7a5\ud558\uba70, \ubd88\uacfc \uc5b8\uc5